In [ ]:
# notebook_snippet.py
import numpy as np
import matplotlib.pyplot as plt

# --- Model ---
def model_U(U, t, r, U_max):
    return r * (U_max - U)

# --- Euler method (manual) ---
def euler_method(func, y0, t_points, params):
    y = np.zeros(len(t_points))
    y[0] = y0
    h = t_points[1] - t_points[0]
    for i in range(len(t_points)-1):
        slope = func(y[i], t_points[i], *params)
        y[i+1] = y[i] + h * slope
    return y

# --- Analytic solution of the model ---
def U_analytic(t, r, U_max, U0):
    return U_max - (U_max - U0)*np.exp(-r * t)

# --- Example synthetic data (ganti dengan load CSV dari Kaggle) ---
# buat data contoh supaya langsung bisa jalan:
t_data = np.arange(0, 30, 1.0)        # days / time units
true_r = 0.08
true_Umax = 10000
U0 = 120
U_true = U_analytic(t_data, true_r, true_Umax, U0)
# tambahkan noise untuk mensimulasikan data nyata
np.random.seed(0)
U_obs = U_true + np.random.normal(scale=150, size=len(U_true))

# --- Estimasi parameter (opsi 1: pakai scipy curve_fit jika tersedia) ---
try:
    from scipy.optimize import curve_fit
    # initial guesses
    p0 = [0.05, U_obs.max()*1.05]  # [r, U_max]
    popt, pcov = curve_fit(lambda t, r, U_max: U_analytic(t, r, U_max, U_obs[0]),
                           t_data, U_obs, p0=p0, maxfev=10000)
    est_r, est_Umax = popt
except Exception as e:
    # fallback sederhana: asumsikan U_max = 1.05*max(obs) dan linearize untuk r
    est_Umax = U_obs.max() * 1.05
    # linearize: ln(U_max - U) = ln(U_max - U0) - r t
    y_lin = np.log(est_Umax - U_obs + 1e-8)  # +small to avoid log(0)
    slope, intercept = np.polyfit(t_data, y_lin, 1)
    est_r = -slope

print("Estimasi: r =", est_r, " U_max =", est_Umax)

# --- Euler simulation with chosen h ---
h = 1.0
t_points = np.arange(t_data[0], t_data[-1] + h, h)
U_euler = euler_method(model_U, U_obs[0], t_points, (est_r, est_Umax))
U_fit_exact = U_analytic(t_points, est_r, est_Umax, U_obs[0])

# --- Error ---
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(U_obs, np.interp(t_data, t_points, U_euler[:len(t_data)]))
print("MSE (data vs Euler interp):", mse)

# --- Plot ---
plt.figure(figsize=(10,6))
plt.plot(t_data, U_obs, 'ko', label='Data Asli (observed)')
plt.plot(t_points, U_fit_exact, 'g-', label='Solusi Eksak Hasil Fit')
plt.plot(t_points, U_euler, 'r--o', label=f'Euler (h={h})')
plt.xlabel('Time')
plt.ylabel('Users / Subscribers')
plt.legend()
plt.grid(True)
plt.show()
